In [5]:
import torch

print(f"CUDA доступна: {torch.cuda.is_available()}")

print(f"PyTorch версия: {torch.__version__}")
print(f"PyTorch cuda версия: {torch.version.cuda}")
print(f"GPU устройство: {torch.cuda.get_device_name(0)}")
print(f"Количество доступных GPU: {torch.cuda.device_count()}")

print(f"Всего памяти GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"Доступно памяти GPU: {torch.cuda.mem_get_info()[0] / 1024**3:.2f} GB")

True
PyTorch версия: 2.5.1+cu124
PyTorch cuda версия: 12.4
GPU устройство: NVIDIA GeForce RTX 3090 Ti
Количество доступных GPU: 1
Всего памяти GPU: 23.99 GB
Доступно памяти GPU: 22.74 GB


## Tест GPU vs CPU

Два типа тестов:
* Матричные операции с большими матрицами (10000x10000)
* Обучение простой нейронной сети на синтетических данных

In [7]:
import torch
import time
import torch.nn as nn
import torch.optim as optim

def matrix_operations_test(device, size=10000):
    """Тест операций с большими матрицами"""
    # Создаем большие матрицы
    matrix1 = torch.randn(size, size, device=device)
    matrix2 = torch.randn(size, size, device=device)
    
    start_time = time.time()
    
    # Выполняем операции
    result = torch.matmul(matrix1, matrix2)
    result = torch.sin(result)
    result = torch.max(result, dim=1)[0]
    
    torch.cuda.synchronize() if device.type == 'cuda' else None
    end_time = time.time()
    
    return end_time - start_time

def neural_network_test(device, num_epochs=10):
    """Тест обучения простой нейронной сети"""
    # Создаем синтетический датасет
    X = torch.randn(100000, 1000, device=device)
    y = torch.randint(0, 2, (100000,), device=device).float()
    
    # Создаем простую модель
    model = nn.Sequential(
        nn.Linear(1000, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 1),
        nn.Sigmoid()
    ).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    
    start_time = time.time()
    
    # Обучаем модель
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs.squeeze(), y)
        loss.backward()
        optimizer.step()
    
    torch.cuda.synchronize() if device.type == 'cuda' else None
    end_time = time.time()
    
    return end_time - start_time

def run_benchmarks():
    # Проверяем доступность CUDA
    device_cpu = torch.device('cpu')
    device_cuda = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"PyTorch версия: {torch.__version__}")
    print(f"CUDA доступна: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Тест матричных операций
    print("\nТест матричных операций:")
    cpu_time = matrix_operations_test(device_cpu)
    print(f"CPU время: {cpu_time:.2f} секунд")
    
    if torch.cuda.is_available():
        gpu_time = matrix_operations_test(device_cuda)
        print(f"GPU время: {gpu_time:.2f} секунд")
        print(f"Ускорение: {cpu_time/gpu_time:.1f}x")
    
    # Тест нейронной сети
    print("\nТест обучения нейронной сети:")
    cpu_time = neural_network_test(device_cpu)
    print(f"CPU время: {cpu_time:.2f} секунд")
    
    if torch.cuda.is_available():
        gpu_time = neural_network_test(device_cuda)
        print(f"GPU время: {gpu_time:.2f} секунд")
        print(f"Ускорение: {cpu_time/gpu_time:.1f}x")


run_benchmarks()

PyTorch версия: 2.5.1+cu124
CUDA доступна: True
GPU: NVIDIA GeForce RTX 3090 Ti

Тест матричных операций:
CPU время: 7.09 секунд
GPU время: 0.39 секунд
Ускорение: 18.2x

Тест обучения нейронной сети:
CPU время: 15.03 секунд
GPU время: 0.16 секунд
Ускорение: 93.2x
